In [ ]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import soundfile as sf
import os
import glob

## tạo file csv cho cơ sở dữ liệu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

tạo file csv cơ sở dữ liệu cho mô hình wakeword


In [ ]:
def generate_csv(folder, label):
    folder_name = os.path.basename(folder)
    labels = list(label)
    print(labels)
    # generate CSV file
    df = pd.DataFrame(columns=["filepath","label"])
    i = 0
    for label in labels:
      print("Reading",os.path.join(folder,label, "*"))
      for filepath in glob.glob(os.path.join(folder,label, "*.wav")):
            df.loc[i] = [filepath,label]
            i += 1
      output_file = f"{folder_name}.csv"
    print("Saving", output_file)
    df.to_csv(output_file)
dir_folder = '/content/drive/MyDrive/wakeword'
generate_csv(dir_folder ,{'lisa oi','noise'})
metadata_filename = "wakeword.csv"
# load CSV files as DataFrames
df0 = pd.read_csv(metadata_filename)
df0 = df0.sample(frac=1).reset_index(drop=True)
df0.tail()

tạo file csv cơ sở dữ liệu cho mô hình nhận dạng câu lệnh

In [ ]:
def generate_csv(folder, gender, label):
    folder_name = os.path.basename(folder)
    labels = list(label)
    genders = list(gender)
    print(labels)
    # generate CSV file
    df = pd.DataFrame(columns=["filepath","label"])
    i = 0
    for label in labels:
      print("Reading",os.path.join(folder,label, "*"))
      for gender in genders:
          for filepath in glob.glob(os.path.join(folder,label, gender, "*.wav")):
            df.loc[i] = [filepath,label]
            i += 1
      output_file = f"{folder_name}.csv"
    print("Saving", output_file)
    df.to_csv(output_file)
dir_folder = '/content/drive/MyDrive/aug_dataset'
generate_csv(dir_folder,{'nu','nam'} ,{'den bat','den tat','den tang sang','den giam sang','den chuyen mau','noise'})
metadata_filename = "command.csv"
# load CSV files as DataFrames
df1 = pd.read_csv(metadata_filename)
df1 = df1.sample(frac=1).reset_index(drop=True)
df1.tail(3)

# tạo hàm trích xuất đặc trưng MFCC

In [ ]:
def get_transform(file):
    audio,rate = librosa.load(file,sr=16000)
    mfccs = librosa.feature.mfcc(y=audio,n_mfcc=13,sr=16000)
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
    features = features.transpose()
    means = np.mean(features, axis=1, keepdims=True)
    stddevs = np.std(features, axis=1, keepdims=True)
    mfcc_feature = (features - means) / (stddevs + 1e-10)
    mfccs_feature = np.zeros((34,39))
    mfccs_feature[:mfcc_feature.shape[0], :mfcc_feature.shape[1]]=mfcc_feature
    return mfccs_feature

# Trích xuất đặc trưng và lưu các đặc trưng

In [ ]:
from tqdm.contrib.concurrent import process_map
feature_mfcc_wakeword = process_map(get_transform, df0['filepath'],max_workers=8)

In [ ]:
feature_mfcc_command = process_map(get_transform, df1['filepath'],max_workers=8)

In [ ]:
def label_command_encode(label):
  if label == 'den bat':
    return 0
  elif label == 'den tat':
    return 1
  elif label == 'den tang sang':
    return 2
  elif label == 'den giam sang':
    return 3
  elif label == 'den chuyen mau':
    return 4
  elif label == 'noise':
    return 5

def label_wakeword_encode(label):
  if label == 'lisa oi':
    return 1
  elif label == 'noise':
    return 0

In [ ]:
df0['label_en'] = df0['label'].apply(label_wakeword_encode)
df1['label_en'] = df1['label'].apply(label_command_encode)

In [ ]:
from tensorflow.keras.utils import to_categorical
split_train_0 = int(len(df0['label_en']) * 0.8)
split_test_0 = int(len(df0['label_en']) * 0.9)
X_train_wakeword = feature_mfcc_wakeword[:split_train_0]
X_val_wakeword = feature_mfcc_wakeword[split_train_0:split_test_0]
X_test_wakeword = feature_mfcc_wakeword[split_test_0:]
y_train_wakeword = np.array(df0['label_en'][:split_train_0].tolist())
y_val_wakeword = np.array(df0['label_en'][split_train_0:split_test_0].tolist())
y_test_wakeword = np.array(df0['label_en'][split_test_0:].tolist())

print(f"Size of the training set: {len(X_train_wakeword)}")
print(f"Size of the test,val set: {len(X_val_wakeword)}")
print(f"Size of the test,val set: {len(X_test_wakeword)}")

In [ ]:
X_train_wakeword_data = np.stack(X_train_wakeword)
X_val_wakeword_data = np.stack(X_val_wakeword)
X_test_wakeword_data = np.stack(X_test_wakeword)
y_test_wakeword_data =np.array(y_test_wakeword)
y_train_wakeword_data =np.array(y_train_wakeword)
y_val_wakeword_data =np.array(y_val_wakeword)

In [ ]:
feature_sets_file0 = '/content/drive/MyDrive/wakeword/wakeword_feature.npz'
np.savez(feature_sets_file0,
         x_train=X_train_wakeword_data,
         y_train=y_train_wakeword_data,
         x_val=X_val_wakeword_data,
         y_val=y_val_wakeword_data,
         x_test=X_test_wakeword_data,
         y_test=y_test_wakeword_data)

In [ ]:
split_train_1 = int(len(df1['label_en']) * 0.8)
split_test_1 = int(len(df1['label_en']) * 0.9)
X_train_command = feature_mfcc_command[:split_train_1]
X_val_command = feature_mfcc_command[split_train_1:split_test_1]
X_test_command = feature_mfcc_command[split_test_1:]
y_train_command = to_categorical(np.array(df1['label_en'][:split_train_1].tolist()))
y_val_command = to_categorical(np.array(df1['label_en'][split_train_1:split_test_1].tolist()))
y_test_command = to_categorical(np.array(df1['label_en'][split_test_1:].tolist()))

print(f"Size of the training set: {len(X_train_command)}")
print(f"Size of the test,val set: {len(X_val_command)}")
print(f"Size of the test,val set: {len(X_test_command)}")

In [ ]:
X_train_command_data = np.stack(X_train_command)
X_val_command_data = np.stack(X_val_command)
X_test_command_data = np.stack(X_test_command)
y_test_command_data =np.array(y_test_command)
y_train_command_data =np.array(y_train_command)
y_val_command_data =np.array(y_val_command)

In [ ]:
feature_sets_file1 = '/content/drive/MyDrive/wakeword/command_feature.npz'
np.savez(feature_sets_file1,
         x_train=X_train_command_data,
         y_train=y_train_command_data,
         x_val=X_val_command_data,
         y_val=y_val_command_data,
         x_test=X_test_command_data,
         y_test=y_test_command_data)